### Аналитикам: большая шпаргалка по Pandas

[Алексей Куличевский](https://smysl.io/blog/pandas/)

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D0%BA%D0%B5%D0%B9%D1%81%D1%8B%20%D0%BF%D0%BE%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D1%83%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/0.%20%D0%9C%D0%B0%D1%80%D0%BA%D0%B5%D1%82%D0%B8%D0%BD%D0%B3.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

Код на чистом Python:

```Python
with open('file.csv') as f:
    content = f.readlines()
    content = [x.split(',').replace('\n','') for x in content]
```

Код, написанный с помощью pandas:

```Python
data = pd.read_csv('file.csv')
```
Pandas добавляет в Python новые структуры данных — серии и датафреймы.

### Структуры данных: серии и датафреймы

Серии — одномерные массивы данных. Они очень похожи на списки, но отличаются по поведению — например, операции применяются к списку целиком, а в сериях — поэлементно.

То есть, если список умножить на 2, получите тот же список, повторенный 2 раза.

In [1]:
# чистый Python
vector = [1, 2, 3]
vector * 2

[1, 2, 3, 1, 2, 3]

А если умножить серию, ее длина не изменится, а вот элементы удвоятся.

In [2]:
import pandas as pd
series = pd.Series([1, 2, 3])
series

0    1
1    2
2    3
dtype: int64

In [3]:
series * 2

0    2
1    4
2    6
dtype: int64

Обратите внимание на первый столбик вывода. Это индекс, в котором хранятся адреса каждого элемента серии. Каждый элемент потом можно получать, обратившись по нужному адресу.

In [4]:
series = pd.Series(['foo', 'bar'])
series

0    foo
1    bar
dtype: object

In [5]:
series[0]

'foo'

Еще одно отличие серий от списков — в качестве индексов можно использовать произвольные значения (похоже на словари), это делает данные нагляднее. Представим, что мы анализируем помесячные продажи. Используем в качестве индексов названия месяцев, значениями будет выручка:

In [6]:
months = ['jan', 'feb', 'mar', 'apr']
sales = [100, 200, 300, 400]

data = pd.Series(data=sales, index=months)
data

jan    100
feb    200
mar    300
apr    400
dtype: int64

Теперь можем получать значения каждого месяца:

In [7]:
data['feb']

200

Так как серии — одномерный массив данных, в них удобно хранить измерения по одному. На практике удобнее группировать данные вместе. Например, если мы анализируем помесячные продажи, полезно видеть не только выручку, но и количество проданных товаров, количество новых клиентов и средний чек. Для этого отлично подходят датафреймы.

Датафреймы — это таблицы. У них есть строки, колонки и ячейки.

Технически, колонки датафреймов — это серии. Поскольку в колонках обычно описывают одни и те же объекты, то все колонки делят один и тот же индекс:

In [8]:
months = ['jan', 'feb', 'mar', 'apr']

# словарь:
sales = {
    'revenue':     [100, 200, 300, 400], # выручка
    'items_sold':  [23, 43, 55, 65], # количество проданных товаров
    'new_clients': [10, 20, 30, 40] # количество новых клиентов
}

sales_df = pd.DataFrame(data=sales, index=months)
sales_df

,revenue,items_sold,new_clients
jan,100,23,10
feb,200,43,20
mar,300,55,30
apr,400,65,40


Бывает, что мы не знаем, что собой представляют данные, и не можем задать структуру заранее. Тогда удобно создать пустой датафрейм и позже наполнить его данными.

In [9]:
df = pd.DataFrame()

А иногда данные уже есть, но хранятся в переменной из стандартного Python, например, в словаре. Чтобы получить датафрейм, эту переменную передаем в ту же команду:

In [10]:
df = pd.DataFrame(data=sales, index=months)

Случается, что в некоторых записях не хватает данных. Например, посмотрите на список `goods_sold` — в нём продажи, разбитые по товарным категориям. 

За первый месяц мы продали машины, компьютеры и программное обеспечение. Во втором машин нет, зато появились велосипеды, а в третьем снова появились машины, но велосипеды исчезли:

In [11]:
goods_sold = [
    {'computers': 10, 'cars': 1, 'soft': 3},
    {'computers': 4, 'soft': 5, 'bicycles': 1},
    {'computers': 6, 'cars': 2, 'soft': 3}
]

Если загрузить данные в датафрейм, Pandas создаст колонки для всех товарных категорий и, где это возможно, заполнит их данными:

In [12]:
pd.DataFrame(goods_sold)

,computers,cars,soft,bicycles
0,10,1.0,3,NaN
1,4,NaN,5,1.0
2,6,2.0,3,NaN


Обратите внимание, продажи велосипедов в первом и третьем месяце равны `NaN` — расшифровывается как `Not a Number`. Так Pandas помечает отсутствующие значения.

Теперь разберем, как загружать данные из файлов. Чаще всего данные хранятся в экселевских таблицах или csv-, tsv- файлах.

Экселевские таблицы читаются с помощью команды `pd.read_excel()`. Параметрами нужно передать адрес файла на компьютере и название листа, который нужно прочитать. Команда работает как с xls, так и с xlsx:

```Python
pd.read_excel('file.xlsx', sheet_name='Sheet1')
```

Файлы формата csv и tsv — это текстовые файлы, в которых данные отделены друг от друга запятыми или табуляцией:

```
# CSV
month,customers,sales
feb,10,200

# TSV
month\tcustomers\tsales
feb\t10\t200
```

Оба читаются с помощью команды `.read_csv()`, символ табуляции передается параметром `sep` (от англ. separator — разделитель):

```Python
pd.read_csv('file.csv')
pd.read_csv('file.tsv', sep='\t')
```

При загрузке можно назначить столбец, который будет индексом. Представьте, что мы загружаем таблицу с заказами. У каждого заказа есть свой уникальный номер, Если назначим этот номер индексом, сможем выгружать данные командой `df[order_id]`. Иначе придется писать фильтр `df[df["id"] == order_id ]`.

Чтобы назначить колонку индексом, добавим в команду `read_csv()` параметр `index_col`, равный названию нужной колонки:

```Python
pd.read_csv('file.csv', index_col='id')
```

После загрузки данных в датафрейм, хорошо бы их исследовать — особенно, если они вам незнакомы.

### Исследуем загруженные данные

Представим, что мы анализируем продажи американского интернет-магазина. У нас есть данные о заказах и клиентах. Загрузим файл с продажами интернет-магазина в переменную `orders`. Раз загружаем заказы, укажем, что колонка `id` пойдет в индекс:

In [13]:
url = 'https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/orders.csv'

orders = pd.read_csv(url, index_col='id')

Расскажу о четырех атрибутах, которые есть у любого датафрейма: `.shape`, `.columns`, `.index` и `.dtypes`.

`.shape` показывает, сколько в датафрейме строк и колонок. Он возвращает пару значений `(n_rows, n_columns)`. Сначала идут строки, потом колонки.

In [14]:
orders.shape

(5009, 4)

В датафрейме 5009 строк и 4 колонки.

Окей, масштаб оценили. Теперь посмотрим, какая информация содержится в каждой колонке. С помощью `.columns` узнаем названия колонок:

In [15]:
orders.columns

Index(['order_date', 'ship_mode', 'customer_id', 'sales'], dtype='object')

Теперь видим, что в таблице есть дата заказа, метод доставки, номер клиента и выручка.

С помощью `.dtypes` узнаем типы данных, находящихся в каждой колонке и поймем, надо ли их обрабатывать. Бывает, что числа загружаются в виде текста. Если мы попробуем сложить две текстовых значения `'1' + '1'`, то получим не число `2`, а строку `'11'`:

In [16]:
orders.dtypes

order_date      object
ship_mode       object
customer_id     object
sales          float64
dtype: object

Тип `object` — это текст, `float64` — это дробное число типа `3,14`.

C помощью атрибута `.index` посмотрим, как называются строки:

In [17]:
orders.index

Int64Index([100006, 100090, 100293, 100328, 100363, 100391, 100678, 100706,
            100762, 100860,
            ...
            167570, 167920, 168116, 168613, 168690, 168802, 169320, 169488,
            169502, 169551],
           dtype='int64', name='id', length=5009)

Ожидаемо, в индексе датафрейма номера заказов: `100762`, `100860` и так далее.

В колонке `sales` хранится стоимость каждого проданного товара. Чтобы узнать разброс значений, среднюю стоимость и медиану, используем метод `.describe()`:

In [18]:
orders.describe()

,sales
count,5009.000000
mean,458.614666
std,954.730953
min,0.556000
25%,37.630000
50%,151.960000
75%,512.064000
max,23661.228000


Наконец, чтобы посмотреть на несколько примеров записей датафрейма, используем команды `.head()` и `.sample()`. Первая возвращает 5 записей из начала датафрейма. Вторая — 5 случайных записей:

In [19]:
orders.head()

,order_date,ship_mode,customer_id,sales
id,,,,
100006,2014-09-07,Standard,DK-13375,377.970
100090,2014-07-08,Standard,EB-13705,699.192
100293,2014-03-14,Standard,NF-18475,91.056
100328,2014-01-28,Standard,JC-15340,3.928
100363,2014-04-08,Standard,JM-15655,21.376


In [20]:
orders.sample(5)

,order_date,ship_mode,customer_id,sales
id,,,,
131884,2015-12-06,Same Day,DK-13375,594.002
160563,2015-10-20,Standard,NS-18640,255.990
162068,2017-12-28,Second,PC-18745,1.188
149895,2017-09-08,Standard,EB-14110,258.528
114517,2014-12-23,Standard,TH-21235,109.836


Получив первое представление о датафреймах, теперь обсудим, как доставать из него данные.

### Получаем данные из датафреймов

Данные из датафреймов можно получать по-разному: указав номера колонок и строк, использовав условные операторы или язык запросов.

Продолжаем анализировать продажи интернет-магазина, которые загрузили в предыдущем разделе. Допустим, я хочу вывести столбец `sales`. Для этого название столбца нужно заключить в квадратные скобки и поставить после них названия датафрейма: `orders['sales']`:

In [21]:
orders['sales']

id
100006     377.970
100090     699.192
100293      91.056
100328       3.928
100363      21.376
            ...   
168802      18.368
169320     171.430
169488      56.860
169502     113.410
169551    1344.838
Name: sales, Length: 5009, dtype: float64

Обратите внимание, результат команды — новый датафрейм с таким же индексом.

Если нужно вывести несколько столбцов, в квадратные скобки нужно вставить список с их названиями: `orders[['customer_id', 'sales']]`. 

Будьте внимательны: квадратные скобки стали двойными. Первые — от датафрейма, вторые — от списка:

In [22]:
orders[['customer_id', 'sales']]

,customer_id,sales
id,,
100006,DK-13375,377.970
100090,EB-13705,699.192
100293,NF-18475,91.056
100328,JC-15340,3.928
100363,JM-15655,21.376
...,...,...
168802,JO-15145,18.368
169320,LH-16900,171.430
169488,AA-10375,56.860


Перейдем к строкам. Их можно фильтровать по индексу и по порядку. Например, мы хотим вывести только заказы `100363`, `100391` и `100706`, для этого есть команда `.loc[]`:

In [23]:
show_these_orders = [100363, 100363, 100706]

In [24]:
orders.loc[show_these_orders]

,order_date,ship_mode,customer_id,sales
id,,,,
100363,2014-04-08,Standard,JM-15655,21.376
100363,2014-04-08,Standard,JM-15655,21.376
100706,2014-12-16,Second,LE-16810,129.440


А в другой раз бывает нужно достать просто заказы с `1` по `3` по порядку, вне зависимости от их номеров в таблицемы. Тогда используют команду `.iloc[]`:

In [25]:
show_these_orders = [1, 2, 3]
orders.iloc[show_these_orders]

,order_date,ship_mode,customer_id,sales
id,,,,
100090,2014-07-08,Standard,EB-13705,699.192
100293,2014-03-14,Standard,NF-18475,91.056
100328,2014-01-28,Standard,JC-15340,3.928


Можно фильтровать датафреймы по колонкам и столбцам одновременно:

In [26]:
columns = ['customer_id', 'sales']
rows = [100363, 100363, 100706]
orders.loc[rows][columns]

,customer_id,sales
id,,
100363,JM-15655,21.376
100363,JM-15655,21.376
100706,LE-16810,129.440


Часто вы не знаете заранее номеров заказов, которые вам нужны. Например, если задача — получить заказы, стоимостью более `1000` рублей. Эту задачу удобно решать с помощью условных операторов.

### Задача

Необходимо узнать, откуда приходят самые большие заказы. Начнем с того, что достанем все покупки стоимостью более 1000 долларов:

In [27]:
filter_large = orders['sales'] > 1000

In [28]:
orders.loc[filter_large]

,order_date,ship_mode,customer_id,sales
id,,,,
101931,2014-10-28,First,TS-21370,1252.602
102673,2014-11-01,Standard,KH-16630,1044.440
102988,2014-04-05,Second,GM-14695,4251.920
103100,2014-12-20,First,AB-10105,1107.660
103310,2014-05-10,Standard,GM-14680,1769.784
...,...,...,...,...
167318,2017-07-26,Standard,GZ-14545,2012.302
167402,2017-01-13,Second,CP-12085,4619.330
167920,2017-12-09,Second,JL-15835,1827.510


Операция `orders['sales'] > 1000` идет по каждому элементу серии и, если условие выполняется, возвращает `True`. Если не выполняется — `False`. Получившуюся серию мы сохраняем в переменную `filter_large`.

Вторая команда фильтрует строки датафрейма с помощью серии. Если элемент `filter_large` равен `True`, заказ отобразится, если `False` — нет. Результат — датафрейм с заказами, стоимостью более `1000` долларов.

In [29]:
# можно без .loc и в одной строчке:
orders[orders['sales'] > 1000]

,order_date,ship_mode,customer_id,sales
id,,,,
101931,2014-10-28,First,TS-21370,1252.602
102673,2014-11-01,Standard,KH-16630,1044.440
102988,2014-04-05,Second,GM-14695,4251.920
103100,2014-12-20,First,AB-10105,1107.660
103310,2014-05-10,Standard,GM-14680,1769.784
...,...,...,...,...
167318,2017-07-26,Standard,GZ-14545,2012.302
167402,2017-01-13,Second,CP-12085,4619.330
167920,2017-12-09,Second,JL-15835,1827.510


Интересно, сколько дорогих заказов было доставлено первым классом? Добавим в фильтр ещё одно условие:

In [30]:
filter_large = orders['sales'] > 1000
filter_first_class = (orders['ship_mode'] == 'First')

orders[filter_large & filter_first_class]

,order_date,ship_mode,customer_id,sales
id,,,,
101931,2014-10-28,First,TS-21370,1252.602
103100,2014-12-20,First,AB-10105,1107.660
106726,2014-12-06,First,RS-19765,1261.330
112158,2014-12-02,First,DP-13165,1050.600
116666,2014-05-08,First,KT-16480,1799.970
...,...,...,...,...
147886,2017-03-28,First,DH-13075,1435.960
155425,2017-11-10,First,AB-10600,1475.054
162558,2017-10-02,First,Dp-13240,2437.672


Логика не изменилась. В переменную `filter_large` сохранили серию, удовлетворяющую условию `orders['sales'] > 1000`. В `filter_first_class` — серию, удовлетворяющую `orders['ship_mode'] == 'First'`.

Затем объединили обе серии с помощью логического `И`: `filter_first_class & filter_first_class`. Получили новую серию той же длины, в элементах которой `True` только у заказов, стоимостью больше `1000`, доставленных первым классом. Таких условий может быть сколько угодно.

Еще один способ решить предыдущую задачу — использовать язык запросов. 

Все условия пишем одной строкой `'sales > 1000 & ship_mode == 'First'` и передаем ее в метод `.query()`. Запрос получается компактнее.

In [31]:
orders.query("sales > 1000 and ship_mode=='First'")

,order_date,ship_mode,customer_id,sales
id,,,,
101931,2014-10-28,First,TS-21370,1252.602
103100,2014-12-20,First,AB-10105,1107.660
106726,2014-12-06,First,RS-19765,1261.330
112158,2014-12-02,First,DP-13165,1050.600
116666,2014-05-08,First,KT-16480,1799.970
...,...,...,...,...
147886,2017-03-28,First,DH-13075,1435.960
155425,2017-11-10,First,AB-10600,1475.054
162558,2017-10-02,First,Dp-13240,2437.672


Значения для фильтров можно сохранить в переменной, а в запросе сослаться на нее с помощью символа `@`: `sales > @sales_filter`.

In [32]:
sales_filter = 1000
ship_mode_filter = 'First'
orders.query('sales > @sales_filter & ship_mode > @ship_mode_filter')

,order_date,ship_mode,customer_id,sales
id,,,,
102673,2014-11-01,Standard,KH-16630,1044.440
102988,2014-04-05,Second,GM-14695,4251.920
103310,2014-05-10,Standard,GM-14680,1769.784
103492,2014-10-10,Standard,CM-12715,1488.774
103660,2014-08-25,Standard,MW-18220,1007.944
...,...,...,...,...
166688,2017-05-20,Standard,RD-19480,1196.316
167318,2017-07-26,Standard,GZ-14545,2012.302
167402,2017-01-13,Second,CP-12085,4619.330


Разобравшись, как получать куски данных из датафрейма, перейдем к тому, как считать агрегированные метрики: количество заказов, суммарную выручку, средний чек, конверсию.

### Считаем производные метрики

Задача: посчитаем, сколько денег магазин заработал с помощью каждого класса доставки. Начнем с простого — просуммируем выручку со всех заказов. Для этого используем метод `.sum()`:

In [33]:
orders['sales'].sum()

2297200.8603000003

Добавим класс доставки. Перед суммированием сгруппируем данные с помощью метода `.groupby()`:

In [34]:
orders.groupby('ship_mode')['sales'].sum()

ship_mode
First       3.514284e+05
Same Day    1.283631e+05
Second      4.591936e+05
Standard    1.358216e+06
Name: sales, dtype: float64

`3.514284e+05` — научный формат вывода чисел. Означает $3.51 * 10^5$. Нам такая точность не нужна, поэтому можем сказать Pandas, чтобы округлял значения до сотых:

In [35]:
pd.options.display.float_format = '{:,.1f}'.format

In [36]:
orders.groupby('ship_mode')['sales'].sum()

ship_mode
First        351,428.4
Same Day     128,363.1
Second       459,193.6
Standard   1,358,215.7
Name: sales, dtype: float64

Другое дело. Теперь видим сумму выручки по каждому классу доставки. По суммарной выручке неясно, становится лучше или хуже. Добавим разбивку по датам заказа:

In [37]:
orders.groupby(['ship_mode', 'order_date'])['sales'].sum()

ship_mode  order_date
First      2014-01-06      12.8
           2014-01-11       9.9
           2014-01-14      62.0
           2014-01-15     149.9
           2014-01-19     378.6
                          ...  
Standard   2017-12-25   1,338.3
           2017-12-26      63.9
           2017-12-28     589.6
           2017-12-29   1,094.8
           2017-12-30     713.8
Name: sales, Length: 2498, dtype: float64

Видно, что выручка прыгает ото дня ко дню: иногда `10` долларов, а иногда `378`. 

Интересно, это меняется количество заказов или средний чек? 

Добавим к выборке количество заказов. Для этого вместо `.sum()` используем метод `.agg()`, в который передадим список с названиями нужных функций.

In [38]:
orders.groupby(['ship_mode', 'order_date'])['sales'].agg(['sum', 'count'])

sum  count
ship_mode order_date               
First     2014-01-06    12.8      1
          2014-01-11     9.9      1
          2014-01-14    62.0      1
          2014-01-15   149.9      1
          2014-01-19   378.6      1
...                      ...    ...
Standard  2017-12-25 1,338.3      7
          2017-12-26    63.9      3
          2017-12-28   589.6      7
          2017-12-29 1,094.8      4
          2017-12-30   713.8      4

[2498 rows x 2 columns]

Ого, получается, что это так прыгает средний чек. Интересно, а какой был самый удачный день? Чтобы узнать, отсортируем получившийся датафрейм: выведем 10 самых денежных дней по выручке:

In [39]:
orders.groupby(['ship_mode', 'order_date'])['sales'].agg(['sum']).sort_values(by='sum', ascending=False).head(10)

sum
ship_mode order_date         
Standard  2014-03-18 26,908.4
          2016-10-02 18,398.2
First     2017-03-23 14,299.1
Standard  2014-09-08 14,060.4
First     2017-10-22 13,716.5
Standard  2016-12-17 12,185.1
          2017-11-17 12,112.5
          2015-09-17 11,467.6
          2016-05-23 10,561.0
          2014-09-23 10,478.6

Параметр `ascending` указывает тип сортировки: по возрастанию или по убыванию.

Команда разрослась, и её теперь неудобно читать. Чтобы упростить, можно разбить её на несколько строк. В конце каждой строки ставим обратный слеш \\:

In [40]:
orders \
    .groupby(['ship_mode', 'order_date'])['sales'] \
    .agg(['sum']) \
    .sort_values(by='sum', ascending=False) \
    .head(10)

sum
ship_mode order_date         
Standard  2014-03-18 26,908.4
          2016-10-02 18,398.2
First     2017-03-23 14,299.1
Standard  2014-09-08 14,060.4
First     2017-10-22 13,716.5
Standard  2016-12-17 12,185.1
          2017-11-17 12,112.5
          2015-09-17 11,467.6
          2016-05-23 10,561.0
          2014-09-23 10,478.6

В самый удачный день — `18 марта 2014 года` — магазин заработал `27 тысяч долларов с помощью стандартного класса доставки`. 

Интересно, откуда были клиенты, сделавшие эти заказы? Чтобы узнать, надо объединить данные о заказах с данными о клиентах.

### Объединяем несколько датафреймов

До сих пор мы смотрели только на таблицу с заказами. Но ведь у нас есть еще данные о клиентах интернет-магазина. Загрузим их в переменную `customers` и посмотрим, что они собой представляют:

In [41]:
url = "https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/customers.csv"

customers = pd.read_csv(url, index_col='id')

In [42]:
customers.head()

,name,segment,state,city
id,,,,
CG-12520,Claire Gute,Consumer,Kentucky,Henderson
DV-13045,Darrin Van Huff,Corporate,California,Los Angeles
SO-20335,Sean O'Donnell,Consumer,Florida,Fort Lauderdale
BH-11710,Brosina Hoffman,Consumer,California,Los Angeles
AA-10480,Andrew Allen,Consumer,North Carolina,Concord


Мы знаем тип клиента, место его проживания, его имя и имя контактного лица. У каждого клиента есть уникальный номер `id`. Этот же номер лежит в колонке `customer_id` таблицы `orders`. Значит мы можем найти, какие заказы сделал каждый клиент. Например, посмотрим, заказы пользователя `CG-12520`:

In [43]:
cust_filter = 'CG-12520'
orders.query('customer_id == @cust_filter')

,order_date,ship_mode,customer_id,sales
id,,,,
152156,2016-11-08,Second,CG-12520,993.9
164098,2017-01-26,First,CG-12520,18.2
123918,2015-10-15,Same Day,CG-12520,136.7


Вернемся к задаче из предыдущего раздела: узнать, что за клиенты, которые сделали `18 марта заказы со стандартной доставкой`. Для этого объединим таблицы с клиентами и заказами. 

Датафреймы объединяют с помощью методов `.concat()`, `.merge()` и `.join()`. Все они делают одно и то же, но отличаются синтаксисом — на практике достаточно уметь пользоваться одним из них.

Покажу на примере `.merge()`:

In [44]:
new_df = pd.merge(orders, customers, 
                  how='inner', 
                  left_on='customer_id', # номер клиента
                  right_index=True) # номер клиента в индексе

In [45]:
new_df.columns

Index(['order_date', 'ship_mode', 'customer_id', 'sales', 'name', 'segment',
       'state', 'city'],
      dtype='object')

In [46]:
new_df.head()

,order_date,ship_mode,customer_id,sales,name,segment,state,city
id,,,,,,,,
100006,2014-09-07,Standard,DK-13375,378.0,Dennis Kane,Consumer,Ohio,Marion
131884,2015-12-06,Same Day,DK-13375,594.0,Dennis Kane,Consumer,Ohio,Marion
145065,2015-12-12,First,DK-13375,32.3,Dennis Kane,Consumer,Ohio,Marion
133046,2017-07-27,Second,DK-13375,298.0,Dennis Kane,Consumer,Ohio,Marion
165099,2017-12-11,First,DK-13375,1.4,Dennis Kane,Consumer,Ohio,Marion


В `.merge()` я сначала указал названия датафреймов, которые хочу объединить. 
Затем уточнил, как именно их объединить и какие колонки использовать в качестве ключа.

Ключ — это колонка, связывающая оба датафрейма. В нашем случае — номер клиента. В таблице с заказами он в колонке `customer_id`, а таблице с клиентами — в индексе. Поэтому в команде мы пишем: `left_on='customer_id', right_index=True`.

### Решаем задачу

Найдем `5 городов, принесших самую большую выручку в 2016 году`.

Для начала отфильтруем заказы из `2016 года`:

In [47]:
orders_2016 = orders.query("order_date >= '2016-01-01' & order_date <= '2016-12-31'")

In [48]:
orders_2016.head()

,order_date,ship_mode,customer_id,sales
id,,,,
100041,2016-11-20,Standard,BF-10975,328.5
100083,2016-11-24,Standard,CD-11980,24.8
100153,2016-12-13,Standard,KH-16630,63.9
100244,2016-09-20,Standard,GM-14695,475.7
100300,2016-06-24,Second,MJ-17740,"4,823.1"


Город — это атрибут пользователей, а не заказов. Добавим информацию о пользователях:

In [49]:
with_customers_2016 = pd.merge(customers, orders_2016, 
                               how='inner', 
                               left_index=True, 
                               right_on='customer_id')

Cруппируем получившийся датафрейм по городам и посчитаем выручку:

In [50]:
grouped_2016 = with_customers_2016.groupby('city')['sales'].sum()

In [51]:
grouped_2016.head()

city
Akron               1,763.0
Albuquerque           692.9
Amarillo              197.2
Arlington           5,672.1
Arlington Heights      14.1
Name: sales, dtype: float64

Отсортируем по убыванию продаж и оставим топ-5:

In [52]:
top5 = grouped_2016.sort_values(ascending=False).head(5)

In [53]:
top5

city
New York City   53,094.1
Philadelphia    39,895.5
Seattle         33,955.5
Los Angeles     33,611.1
San Francisco   27,990.0
Name: sales, dtype: float64

### Упражнение:

1. Сколько заказов, отправлено первым классом за последние 5 лет?
2. Сколько в базе клиентов из Калифорнии?
3. Сколько заказов они сделали?
4. Постройте сводную таблицу средних чеков по всем штатам за каждый год.